<a href="https://colab.research.google.com/github/stanley1208/AI-Learning/blob/main/Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms

In [18]:
# 1. Transforms (convert images to tensors + normalize)
transform=transforms.Compose([
    transforms.ToTensor(),
  ])

# 2. Load CIFAR-10 (train set)
train_data=datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
test_data=datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)

# 3. Wrap in DataLoader
train_loader=DataLoader(train_data,batch_size=32,shuffle=True)
test_loader=DataLoader(test_data,batch_size=32,shuffle=True)



In [ ]:
# Create a tiny fake dataset: 100 samples, each 10-dimensional
X=torch.randn(100,3,32,32)
y=torch.randint(0,2,(100,)) # binary labels (0 or 1)

dataset=list(zip(X,y))
dataloader=DataLoader(dataset,batch_size=16,shuffle=True)



In [ ]:
class TinyModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc=nn.Linear(10,2) # input: 10 dims, output: 2 classes

  def forward(self,x):
    return self.fc(x)

In [23]:
model=TinyCNN() # loss function
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.01)  # optimizer

In [19]:
def train_one_model(model,dataloader,optimiser,crtierion):
  model.train()
  total_loss=0

  for x_batch, y_batch in dataloader:
      # 0️⃣ Reset gradients
      optimizer.zero_grad()
      # 1️⃣ Forward pass
      preds=model(x_batch)  # shape: [batch_size, 2]
      # 2️⃣ Compute loss
      loss=criterion(preds,y_batch) # scalar
      # 3️⃣ Backward pass
      loss.backward()
      # 4️⃣ Optimizer step
      optimizer.step()

      total_loss+=loss.item()


  avg_loss=total_loss/len(dataloader)
  return avg_loss

In [20]:
def evaluate(model,dataloader,criterion):
  model.eval()  # switch to eval mode
  correct=0
  total=0
  total_loss=0.0

  with torch.no_grad():   # no gradients needed for eval
    for x_batch,y_batch in dataloader:
      preds=model(x_batch)  # forward only
      loss=criterion(preds,y_batch)
      predicted=preds.argmax(dim=1) # class with highest score

      correct+=(predicted==y_batch).sum().item()
      total+=y_batch.size(0)
      total_loss+=loss.item()

  avg_loss=total_loss/len(dataloader)
  accuracy=correct/total
  return avg_loss,accuracy

Epoch 1 | Train Loss: 0.7317 | Val Loss: 0.6991 | Val Acc: 0.5400
Epoch 2 | Train Loss: 0.6919 | Val Loss: 0.6934 | Val Acc: 0.5300
Epoch 3 | Train Loss: 0.7030 | Val Loss: 0.6864 | Val Acc: 0.5400


In [21]:
class TinyCNN(nn.Module):
  def __init__(self):
    super().__init__()
    # conv1: in_channels=3 (RGB), out_channels=16, kernel=3, padding=1 keeps spatial size
    self.conv1=nn.Conv2d(3,16,kernel_size=3,padding=1)
    self.bn1=nn.BatchNorm2d(16)
    # add maxpool
    self.pool=nn.MaxPool2d(kernel_size=2,stride=2)

    # conv2: in_channels=16, out_channels=32
    self.conv2=nn.Conv2d(16,32,kernel_size=3,padding=1)
    self.bn2=nn.BatchNorm2d(32)
    # global pooling to reduce spatial dims
    self.gap=nn.AdaptiveAvgPool2d((1,1))
    # final linear layer: 32 channels -> 2 classes (match your toy labels)
    self.fc=nn.Linear(32,10)

  def forward(self,x):
    # [B,3,32,32] -> [B,16,32,32] -> [B,16,16,16]
    x=self.pool(F.relu(self.bn1(self.conv1(x)))) # [B,16,H,W]
    # [B,16,16,16] -> [B,32,16,16]
    x=F.relu(self.bn2(self.conv2(x))) # [B,32,H,W]
    # [B,32,16,16] -> [B,32,1,1]
    x=self.gap(x)
    # [B,32,1,1] -> [B,32]        # [B,32,1,1]
    x=x.view(x.size(0),-1)
    # [B,32] -> [B,10]        # [B,32]
    out=self.fc(x)                    # [B,2]

    return out


In [21]:
model=TinyCNN()
print(model)

TinyCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc): Linear(in_features=32, out_features=10, bias=True)
)


In [14]:
x=torch.randn(16,3,32,32)
out=model(x)
print(out.shape)

torch.Size([16, 10])


In [15]:
model=TinyCNN()
x=torch.randn(8,3,32,32)
out=model(x)
print(out.shape)

torch.Size([8, 10])


In [24]:
num_epoch=3
for epoch in range(num_epoch):
  train_loss=train_one_model(model,train_loader,optimizer,criterion)
  val_loss,val_accuracy=evaluate(model,test_loader,criterion)

  print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_accuracy:.4f}")

Epoch 1 | Train Loss: 2.0168 | Val Loss: 1.9414 | Val Acc: 0.2866
Epoch 2 | Train Loss: 1.7908 | Val Loss: 1.8540 | Val Acc: 0.3159
Epoch 3 | Train Loss: 1.6728 | Val Loss: 2.5898 | Val Acc: 0.1798


In [ ]:
for xb,yb in train_loader:
  print("xb:",xb.shape)
  print("yb:",yb.shape)
  break

xb: torch.Size([32, 3, 32, 32])
yb: torch.Size([32])
